In [13]:
MODEL = "gemma2:2b"

In [19]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [20]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/11" 200 None


"Why don't scientists trust atoms? \n\nBecause they make up everything! 😉 \n\n\nLet me know if you want to hear another one! 😄 \n"

In [21]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/dm1.pdf")
pages = loader.load_and_split()

pages

[Document(metadata={'source': 'data/dm1.pdf', 'page': 0}, page_content='MODULE  1 \nSYSTEMS  OF EARTH  AND  KEY  TERMS  IN DISASTER  \nMANAGEMENT  \n \n \n \nSYLLABUS:  \nSystems of earth: Lithosphere - composition, rocks, soils; Atmosphere -layers, ozone layer, greenhouse \neffect, weather, cyclones, atmospheric circulations, Indian Monsoon; hydrosphere Oceans, inland water \nbodies; biosphere.  \n \nDefinition and meaning of key terms in Disaster Risk Reduction and Management - disaster, hazard, \nexposure, vulnerability, risk, risk assessment, risk mapping, capacity, resilience, disaster risk reduction, \ndisaster risk management, early warning systems, disaster preparedness, disaster prevention, disaster \nmitigation, disaster response, damage assessment, crisis counselling, needs assessment.'),
 Document(metadata={'source': 'data/dm1.pdf', 'page': 1}, page_content='DISASTER  MANAGEMENT  \n \n \nDisaster management refers to the conservation of lives and property  during natural or

In [22]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I'm not sure".

Context : {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context = "Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t answer the question, reply "I\'m not sure".\n\nContext : Here is some context\n\nQuestion: Here is a question\n'

In [23]:
chain = prompt | model 

In [24]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [25]:
chain.invoke(
    {
        "context": "The name I was given was San",
        "question": "What is my name?"
    }
)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/11" 200 None


'San \n'

In [26]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

INFO - docarray - DB config created
INFO:docarray:DB config created
INFO - docarray - Runtime config created
INFO:docarray:Runtime config created
INFO - docarray - No docs or index file provided. Initializing empty InMemoryExactNNIndex.
INFO:docarray:No docs or index file provided. Initializing empty InMemoryExactNNIndex.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 2

In [27]:
retriever = vectorstore.as_retriever()

retriever.invoke("Disaster Management")

DEBUG:langchain_core.vectorstores.base:The embeddings property has not been implemented for DocArrayInMemorySearch
DEBUG:langchain_core.vectorstores.base:The embeddings property has not been implemented for DocArrayInMemorySearch
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG - docarray - Executing `find` for search field embedding
DEBUG:docarray:Executing `find` for search field embedding


[Document(metadata={'source': 'data/dm1.pdf', 'page': 26}, page_content='NEEDS  ASSESSMENT  \n \nNeeds assessment is a process of estimating the financial, technical, and human resources needed to \nimplement  the agreed -upon  programmes  of recovery,  reconstruction,  and risk management . Post-damage \nneeds assessment  is normally a rapid, multi -sectoral assessment  that measures the impact of disasters on \nthe society, economy, and environment of the disaster -affected areas.  \n \n \n \nDISASTER  MANAGEMENT  CYCLE  \n \nDisaster  Risk Management  includes  sum total of all activities,  programs  and measures  which  can be taken \nup before, during and after a disaster  with the purpose to avoid a disaster , reduce its impact  or recover \nfrom its losses.  \n \nThe steps  involved  in Disaster  management  Cycle  \n \n \nThe different  approaches  involved  are: \na) Before  A Disaster  (Pre-Disaster):  \nActivities  taken  to reduce  human  and property  losses  caused  by a 

In [28]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [30]:
print({chain.invoke({'question': "what is disaster managment"})})

DEBUG:langchain_core.vectorstores.base:The embeddings property has not been implemented for DocArrayInMemorySearch


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/11" 200 None
DEBUG - docarray - Executing `find` for search field embedding
DEBUG:docarray:Executing `find` for search field embedding
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
